In [1]:
import os
import numpy as np
import pandas as pd
from tensorflow import keras
from keras.applications import MobileNetV2
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Model
from sklearn.metrics.pairwise import cosine_similarity 

In [2]:
def extract_feature_vector_v3(image_file):
    
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(512, 512, 3), pooling='avg')
    model = tf.keras.Model(inputs=base_model.input, outputs=base_model.output)

    try:
        image_data = BytesIO(image_file.read())  # Read file-like object
        img = Image.open(image_data)
        
    except UnidentifiedImageError:
        st.error("The uploaded file is not a valid image. Please upload a valid image file.")
        return None

    # Convert image to JPEG if not already
    if img.format != 'JPEG':
        img = img.convert("RGB")  # Convert to RGB for JPEG compatibility
        image_data = BytesIO()  # Reset the file-like object
        img.save(image_data, format='JPEG')  # Save as JPEG
        image_data.seek(0)  # Reset the pointer to the beginning

    img = img.resize((512, 512))  # Resize to 512x512 images
    img_array = img_to_array(img)
    if img_array.shape != (512, 512, 3):
        st.error("Image has incorrect shape.")
        return None
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)  # Preprocess for MobileNetV2

    if img_array.dtype != np.float32:
        st.error("Input data type is not float32.")
        return None

    feature_vector = model.predict(img_array).flatten()  # Flatten to get a 1D vector
    return feature_vector